## Exploración y entendimiento profundo de los datos ##

Librerias:

In [13]:
import pandas as pd
import zipfile
import os
import numpy as np

Datos:

In [14]:
cd "/Users/monserratarredondo/GitHub/Prediccion_de-_demanda_en_retail"

/Users/monserratarredondo/GitHub/Prediccion_de-_demanda_en_retail


In [15]:
#PATH = "~/"
PATH = "/Users/monserratarredondo/GitHub/Prediccion_de-_demanda_en_retail/Data/"
#/Users/monserratarredondo/GitHub/Prediccion_de-_demanda_en_retail/Data/

item_categories = pd.read_csv(PATH + "item_categories.csv", encoding='utf-8', low_memory=False)
items= pd.read_csv(PATH + "items.csv", encoding='utf-8', low_memory=False)
sales_train = pd.read_csv(PATH + "sales_train.csv", encoding='utf-8', low_memory=False)
sample_submission = pd.read_csv(PATH + "sample_submission.csv", encoding='utf-8', low_memory=False)
shops = pd.read_csv(PATH + "shops.csv", encoding='utf-8', low_memory=False)
test = pd.read_csv(PATH + "test.csv", encoding='utf-8', low_memory=False)

In [16]:
item_categories.info()

<class 'pandas.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   item_category_name  84 non-null     str  
 1   item_category_id    84 non-null     int64
dtypes: int64(1), str(1)
memory usage: 1.4 KB


In [19]:
sample_submission.info()

<class 'pandas.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              214200 non-null  int64  
 1   item_cnt_month  214200 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 3.3 MB


In [20]:
shops.info()

<class 'pandas.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   shop_name  60 non-null     str  
 1   shop_id    60 non-null     int64
dtypes: int64(1), str(1)
memory usage: 1.1 KB


In [21]:
test.info()

<class 'pandas.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB


*Exploración sobre datos históricos de ventas diarias (2013-2015)*

In [23]:
sales_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [24]:
sales_train.head(3)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0


*Combinaciones producto-tienda para predicción*

In [25]:
test.head(3)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233


*Información de productos y categorías*

In [26]:
items.head(3)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40


Transformación y preparación de features

In [ ]:
for df, col in [(sales,"shop_id"), (sales,"item_id"),
                (test,"shop_id"), (test,"item_id"),
                (items,"item_id"), (items,"item_category_id"),
                (shops,"shop_id"), (cats,"item_category_id")]:
    df[col] = df[col].astype("int32")


In [ ]:
sales["date_block_num"] = sales["date"].dt.to_period("M").astype(str)  # o numérico
# Si tienes un date_block_num ya en el dataset original, úsalo.

monthly = (
    sales
    .groupby(["shop_id","item_id", pd.Grouper(key="date", freq="MS")], as_index=False)
    .agg(
        item_cnt_day_sum=("item_cnt_day", "sum"),
        item_price_mean=("item_price", "mean")
    )
    .rename(columns={"date":"month"})
)


In [ ]:
test_enriched = (
    test
    .merge(items_dim, on="item_id", how="left")
    .merge(shops, on="shop_id", how="left")
)


In [ ]:
hist_feat = (
    sales.groupby(["shop_id","item_id"], as_index=False)
    .agg(
        hist_mean_cnt=("item_cnt_day", "mean"),
        hist_sum_cnt=("item_cnt_day", "sum"),
        hist_mean_price=("item_price", "mean"),
    )
)

test_final = test_enriched.merge(hist_feat, on=["shop_id","item_id"], how="left")
